In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn

In [4]:
from skorch import NeuralNetClassifier

X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=nn.ReLU()):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, num_units)
        self.output = nn.Linear(num_units, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X

In [5]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=50,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

In [6]:
net.fit(X, y)
y_proba = net.predict_proba(X)
print(y_proba, "is calculated probability.")

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7105       0.4900        0.6951  0.0105
      2        0.6940       0.5350        0.6838  0.0077
      3        0.6849       0.5750        0.6764  0.0072
      4        0.6697       0.6200        0.6714  0.0074
      5        0.6696       0.6300        0.6664  0.0074
      6        0.6469       0.6550        0.6583  0.0077
      7        0.6359       0.6450        0.6501  0.0075
      8        0.6258       0.6650        0.6410  0.0076
      9        0.6132       0.6500        0.6285  0.0068
     10        0.6221       0.6750        0.6216  0.0080
     11        0.6043       0.6900        0.6127  0.0074
     12        0.5759       0.7100        0.6019  0.0065
     13        0.5983       0.7250        0.5952  0.0074
     14        0.5962       0.7050        0.5871  0.0069
     15        0.5784       0.6900        0.5804  0.0067
     16        0.5866       0.7